# Loop 20 Strategic Analysis

## Current Status
- **Best CV score**: 70.316579 (from 020_optimal_whynot_ensemble)
- **Best LB score**: 70.3434 (from exp_019)
- **Target**: 68.876781
- **Gap**: 1.44 points (2.09%)

## Key Insight
The current submission (70.316579) is BETTER than all external kernel outputs:
- bbox3-ensemble-update: 70.319731
- base-model-ensemble: 70.331130
- fork-of-the-fork: 70.331169
- team-optimization-blend: 70.331635

This means we have the best publicly available ensemble!

In [1]:
import pandas as pd
import numpy as np
from numba import njit
import math
import json

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs,ys,degs,tx,ty):
    n=xs.size; V=tx.size
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r=degs[i]*math.pi/180.0
        c=math.cos(r); s=math.sin(r)
        xi=xs[i]; yi=ys[i]
        for j in range(V):
            X=c*tx[j]-s*ty[j]+xi
            Y=s*tx[j]+c*ty[j]+yi
            if X<mnx: mnx=X
            if X>mxx: mxx=X
            if Y<mny: mny=Y
            if Y>mxy: mxy=Y
    side=max(mxx-mnx,mxy-mny)
    return side*side/n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a],np.float64)

tx, ty = make_polygon_template()

# Load current submission
df = pd.read_csv('/home/submission/submission.csv')
df['N'] = df['id'].str.split('_').str[0].astype(int)

# Calculate per-N scores
per_n_scores = {}
for n in range(1, 201):
    g = df[df['N'] == n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    per_n_scores[n] = score_group(xs, ys, ds, tx, ty)

total = sum(per_n_scores.values())
print(f"Current submission total: {total:.6f}")
print(f"Target: 68.876781")
print(f"Gap: {total - 68.876781:.6f} ({(total - 68.876781) / 68.876781 * 100:.2f}%)")

# Show top 20 N values by score contribution
print("\nTop 20 N values by score contribution:")
for n, score in sorted(per_n_scores.items(), key=lambda x: -x[1])[:20]:
    print(f"  N={n}: {score:.6f}")

Current submission total: 70.316579
Target: 68.876781
Gap: 1.439798 (2.09%)

Top 20 N values by score contribution:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399842
  N=6: 0.399610
  N=8: 0.385407
  N=9: 0.383047
  N=10: 0.376630
  N=11: 0.374921
  N=15: 0.374381
  N=12: 0.372724
  N=13: 0.372267
  N=20: 0.371795
  N=16: 0.370191
  N=17: 0.370040
  N=22: 0.369818
  N=14: 0.369543
  N=33: 0.369347


In [2]:
# Analyze what improvements are needed
print("\n=== Gap Analysis ===")
print(f"Current: {total:.6f}")
print(f"Target: 68.876781")
print(f"Gap: {total - 68.876781:.6f}")

# If we improved each N by the same amount, how much would we need?
avg_improvement_needed = (total - 68.876781) / 200
print(f"\nAverage improvement needed per N: {avg_improvement_needed:.6f}")

# What percentage improvement is needed?
pct_improvement = (total - 68.876781) / total * 100
print(f"Percentage improvement needed: {pct_improvement:.2f}%")

# How many N values would need to improve by 0.01 each?
num_n_needed = (total - 68.876781) / 0.01
print(f"\nIf each N improved by 0.01: need {num_n_needed:.0f} N values")

# How many N values would need to improve by 0.1 each?
num_n_needed_01 = (total - 68.876781) / 0.1
print(f"If each N improved by 0.1: need {num_n_needed_01:.0f} N values")


=== Gap Analysis ===
Current: 70.316579
Target: 68.876781
Gap: 1.439798

Average improvement needed per N: 0.007199
Percentage improvement needed: 2.05%

If each N improved by 0.01: need 144 N values
If each N improved by 0.1: need 14 N values


In [3]:
# Check which N values have the most room for improvement
# Compare to theoretical minimum (single tree bounding box)
print("\n=== Theoretical Analysis ===")

# For N=1, the optimal score is the bounding box of a single tree
# Tree dimensions: width=0.7, height=1.0 (from -0.2 to 0.8)
# Optimal rotation: 45 degrees gives side = sqrt(0.7^2 + 1.0^2) / sqrt(2) = 0.86
# Actually, let's compute it properly

import numpy as np

# Tree vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def compute_bbox_for_angle(angle_deg):
    rad = angle_deg * np.pi / 180
    c, s = np.cos(rad), np.sin(rad)
    xs = c * TX - s * TY
    ys = s * TX + c * TY
    width = xs.max() - xs.min()
    height = ys.max() - ys.min()
    side = max(width, height)
    return side * side

# Find optimal angle for N=1
best_angle = 0
best_score = float('inf')
for angle in range(0, 360):
    score = compute_bbox_for_angle(angle)
    if score < best_score:
        best_score = score
        best_angle = angle

print(f"N=1 theoretical optimal: angle={best_angle}°, score={best_score:.6f}")
print(f"N=1 current: {per_n_scores[1]:.6f}")
print(f"N=1 gap: {per_n_scores[1] - best_score:.6f}")

# The N=1 score is already optimal (0.6612 = 0.813^2)


=== Theoretical Analysis ===
N=1 theoretical optimal: angle=45°, score=0.661250
N=1 current: 0.661250
N=1 gap: -0.000000


In [4]:
# Check submission history
print("\n=== Submission History ===")
with open('/home/code/session_state.json') as f:
    state = json.load(f)

print(f"Submissions used: {state['max_submissions'] - state['remaining_submissions']}")
print(f"Remaining: {state['remaining_submissions']}")

print("\nExperiments with LB scores:")
for exp in state['experiments']:
    lb = exp.get('lb_score')
    cv = exp.get('cv_score', exp.get('score'))
    if lb:
        print(f"  {exp['id']}: CV={cv:.4f}, LB={lb}")

print("\nBest experiments by CV:")
sorted_exps = sorted(state['experiments'], key=lambda x: x.get('cv_score', x.get('score', 999)))
for exp in sorted_exps[:5]:
    cv = exp.get('cv_score', exp.get('score'))
    print(f"  {exp['id']} ({exp['name']}): CV={cv:.4f}")


=== Submission History ===
Submissions used: 1
Remaining: 99

Experiments with LB scores:

Best experiments by CV:
  exp_007 (007_ensemble_fractional): CV=70.2657
  exp_009 (009_highprec_ensemble): CV=70.3411
  exp_013 (013_selective_threshold): CV=70.3421
  exp_019 (019_comprehensive_external_ensemble): CV=70.3434
  exp_016 (016_mega_ensemble_external): CV=70.3535
